# Converting interchangeably between crystal structure and a grayscale PNG image (basic `xtal2png` tutorial)

In this notebook, we will install the `xtal2png` package, encode/decode two example pymatgen `Structure` objects, and show some visualizations of the intermediate PNG representations and before/after crystal structure plots. Finally, we comment on how you can use `xtal2png` with state-of-the-art machine learning image models.

## Installation

Install the `xtal2png` package. Optionally install `ase` and `nglview` which can be used to visualize crystal structures. You may need to restart the runtime via `Ctrl+M, .` or `Runtime --> Restart Runtime` (via menubar).

In [ ]:
!pip install xtal2png
!pip install ase nglview # optional, for visualization of crystal structures

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 40.6 MB 2.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 65 kB 3.9 MB/s 
     |████████████████████████████████| 98 kB 6.4 MB/s 
     |████████████████████████████████| 109 kB 50.8 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 325 kB 64.7 MB/s 
     |████████████████████████████████| 546 kB 52.2 MB/s 


## Encode(/decode) two pymatgen `Structure` objects

Import a list of two example pymatgen `Structure` objects (these correspond to [mp-560471](https://next-gen.materialsproject.org/materials/mp-560471)/$Zn_2B_2PbO_6$ and [mp-7823](https://next-gen.materialsproject.org/materials/mp-7823)/$V_2NiSe_4$, respectively)

In [ ]:
from xtal2png.utils.data import example_structures
from xtal2png.core import XtalConverter

Let's take a look at the second `Structure` which has a smaller footprint.

In [ ]:
example_structures[1]

We will be using the `XtalConverter` class, for which more information including its `__init__` arguments and functions can be displayed via `help(XtalConverter)`. For just the parameters for class instantiation, try `help(XtalConverter.__init__)`. Note that `max_sites` is not tested for values other than `52`.

In [ ]:
help(XtalConverter.__init__)

Let's specify the save directory (`save_dir`) for the PNG files as `"data"`, which will be automatically created. In this case, it will be saved to temporary Google Colab storage.

In [ ]:
xc = XtalConverter(save_dir="data") # DFT surrogate relaxation via m3gnet by default
data = xc.xtal2png(example_structures, save=True)
relaxed_decoded_structures = xc.png2xtal(data, save=False)

We also take a look at the unrelaxed decoded structures.

In [ ]:
xc = XtalConverter(save_dir="data", relax_on_decode=False)
data = xc.xtal2png(example_structures, save=True)
decoded_structures = xc.png2xtal(data, save=False)

## Visualization

For visualization, we'll cover two aspects: the structure-encoded PNG images and visualizing before/after crystal structures.

### Structure-encoded PNG images

Note that images won't show via `im.show()` command _on Google Colab_ even if you specify `xc.xtal2png(..., show=True, ...)`, so for this Colab example we'll open the images ad-hoc based on where they were saved in local Colab storage. We display the images stacked one on top of another using `display(im)` instead of `im.show()`. Note that the filepaths have the chemical formula, `volume`, and a randomly generated `uid` portion to promote uniqueness, especially when dealing with allotropes (same chemical formula, different crystal structure).

In [ ]:
import glob, os
from PIL import Image
for fpath in glob.glob("data/*.png"):
    with Image.open(fpath) as im:
      im = im.resize((64*5, 64*5), Image.BOX)
      print(fpath)
      display(im)

As mentioned in the README, the legend key for these images is as follows:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<img src=https://user-images.githubusercontent.com/45469701/169937021-f6f60169-6965-4db1-9bbd-e8744521d570.png width=327>

Also described in the README, the match between the encoded and decoded versions is within an expected tolerance, given that PNG images are represented as discrete RGB values between 0 and 255 (i.e. there is a round-off error).

<table>
<tr>
<th> Original </th>
<th> Decoded </th>
<th> Relaxed Decoded </th>
</tr>
<tr>
<td>

```python
Structure Summary
Lattice
    abc : 5.033788 11.523021 10.74117
 angles : 90.0 90.0 90.0
 volume : 623.0356027127609
      A : 5.033788 0.0 3.0823061808931787e-16
      B : 1.8530431062799525e-15 11.523021 7.055815392078867e-16
      C : 0.0 0.0 10.74117
PeriodicSite: Zn2+ (0.9120, 5.7699, 9.1255) [0.1812, 0.5007, 0.8496]
PeriodicSite: Zn2+ (4.1218, 5.7531, 1.6156) [0.8188, 0.4993, 0.1504]
...
```

</td>
<td>

```python
Structure Summary
Lattice
    abc : 5.0250980392156865 11.533333333333331 10.8
 angles : 90.0 90.0 90.0
 volume : 625.9262117647058
      A : 5.0250980392156865 0.0 0.0
      B : 0.0 11.533333333333331 0.0
      C : 0.0 0.0 10.8
PeriodicSite: Zn (0.9016, 5.7780, 3.8012) [0.1794, 0.5010, 0.3520]
PeriodicSite: Zn (4.1235, 5.7554, 6.9988) [0.8206, 0.4990, 0.6480]
...
```

</td>
<td>

```python
Structure Summary
Lattice
    abc : 5.026834307381214 11.578854613685237 10.724087971087924
 angles : 90.0 90.0 90.0
 volume : 624.1953646135236
      A : 5.026834307381214 0.0 0.0
      B : 0.0 11.578854613685237 0.0
      C : 0.0 0.0 10.724087971087924
PeriodicSite: Zn (0.9050, 5.7978, 3.7547) [0.1800, 0.5007, 0.3501]
PeriodicSite: Zn (4.1218, 5.7810, 6.9693) [0.8200, 0.4993, 0.6499]
...
```

</td>
</tr>
</table>

### Before/after Crystal Structure Visualization

To visualize the crystal structures before and after, we can use `nglview` after a bit of Colab finnagling with external ipywidgets.

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.visualize import view
aaa = AseAtomsAdaptor()

In [ ]:
[display(view(aaa.get_atoms(s), viewer='ngl')) for s in example_structures]

In [ ]:
[display(view(aaa.get_atoms(s), viewer='ngl')) for s in decoded_structures]

In [ ]:
[display(view(aaa.get_atoms(s), viewer='ngl')) for s in relaxed_decoded_structures]

Undo the Colab finnagling of external ipywidgets.

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

## Final Remarks

This tool makes it possible to use state-of-the-art image-based machine learning models with minimal "plumbing" required. Just follow the normal instructions for custom image datasets. For example, this can be used with [Palette](https://iterative-refinement.github.io/palette/), an image-to-image guided diffusion model by Google, which has an unofficial implementation [here](https://github.com/Janspiry/Palette-Image-to-Image-Diffusion-Models).